In [45]:
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
STOPWORDS = thai_stopwords()
from glob import glob
from tqdm import tqdm
import json, html, re
import numpy as np
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
import pandas as pd

jsons = glob('tweets/*.json')
for i, f in enumerate(jsons):
    print(i,f)
    
def clean(text):
    text = html.unescape(text)
    text = re.sub(r'http.+?(?:\\s|$)', '', text) # URL link
    text = re.sub(r'[“”„]', '"', text) # convert double quotations into "
    text = re.sub(r'[‘’′′′′`]', "'", text) # convert single quotations into '
    text = re.sub(r'[ \u00a0\xa0\u3000\u2002-\u200a\t\n#]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
    text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
    text = re.sub(r'ํา','ำ', text) # am
    return text.strip()

def tokenize(text):
    tokens = word_tokenize(clean(text), keep_whitespace='False')
    tokens = [token for token in tokens if token not in STOPWORDS and re.match(r'[ก-๙][ก-๙\\.\\-]+$', token)]
    return tokens

0 tweets/#กูสั่งให้มึงอยู่ใต้รัฐธรรมนูญ.json
1 tweets/#ธรรมศาสตร์และการชุมนุม.json
2 tweets/#เกียมอุดมไม่ก้มหัวให้เผด็จการ.json
3 tweets/#25พฤศจิกาไปSCB.json
4 tweets/#ม็อบ25พฤศจิกาทวงคืนสมบัติชาติ.json
5 tweets/#ม็อบ2ธันวา.json


In [32]:
df = pd.read_json(jsons[3], lines=True)
df['tokens'] = df.tweet.apply(tokenize)

In [33]:
df.tokens

0        [ชวน, ทบทวน, ม็อบ, พฤศจิกา, พฤศจิกา, ประเด็น, ...
1        [วาน, แอมป์, วรรษ, รับทราบ, ข้อหา, ร้องเพลง, ฉ...
2        [วรรษ, เลี้ยง, วัฒนา, บัณฑิต, ม., ศิลปากร, หมา...
3                                                [พฤศจิกา]
4                                                [พฤศจิกา]
                               ...                        
49045                                            [พฤศจิกา]
49046    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
49047                                       [คับ, พฤศจิกา]
49048    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
49049    [ม็อบ, พฤศจิกา, เปลี่ยนไป, สำนักงานใหญ่, ลด, ก...
Name: tokens, Length: 49050, dtype: object

In [42]:
corpus_dictionary = Dictionary(df.tokens)
corpus = []
for tokens_list in tqdm(df.tokens):
    corpus.append(corpus_dictionary.doc2bow(tokens_list))
lda = LdaModel(corpus, num_topics=5, id2word=corpus_dictionary, passes=10)

100%|██████████| 49050/49050 [00:00<00:00, 87367.57it/s]


In [44]:
for i in range(10):
    print(f'|{i+1}', end='')
    for j in range(5):
        word, score = lda.show_topic(j)[i]
        print(f"|{word}|{score:.3f}",end="")
    print('|')

|1|พฤศจิกา|0.071|พฤศจิกา|0.284|พฤศจิกา|0.054|พฤศจิกา|0.086|พฤศจิกา|0.095|
|2|ชุมนุม|0.025|ม็อบ|0.140|คน|0.025|นะคะ|0.064|ตำรวจ|0.048|
|3|ตู้|0.016|รัฐประหาร|0.026|ทำ|0.014|อย่า|0.029|อี|0.026|
|4|รถ|0.015|แกง|0.025|กลัว|0.012|ม็อบ|0.020|ทำ|0.024|
|5|ข่าว|0.014|ต่อต้าน|0.021|รู้|0.010|คน|0.019|ทหาร|0.023|
|6|ถนน|0.014|รถติด|0.016|เค้า|0.010|ช่วยกัน|0.016|ประชาชน|0.022|
|7|ย้าย|0.013|พฤศจิกายน|0.008|ดี|0.008|ขอให้|0.013|เหี้ย|0.019|
|8|คอนเทนเนอร์|0.013|กก|0.008|เรื่อง|0.008|ใส่|0.011|ขนาด|0.014|
|9|ติด|0.013|ราษฎร|0.008|ม๊อบ|0.007|เกรียน|0.010|หรอ|0.013|
|10|ประกาศ|0.012|ชุมนุม|0.007|สลิ่ม|0.007|ระวัง|0.010|ขน|0.013|


In [46]:
df.tokens.apply(len).mean()

9.77875637104995

In [47]:
df.tokens[0]

['ชวน',
 'ทบทวน',
 'ม็อบ',
 'พฤศจิกา',
 'พฤศจิกา',
 'ประเด็น',
 'ทรัพย์สินส่วนพระมหากษัตริย์',
 'สรุป',
 'ส่วนรวม',
 'อ่าน']